<a href="https://colab.research.google.com/github/GDS-Education-Community-of-Practice/DSECOP/blob/main/Connecting_MonteCarlo_to_ModernAI/04_Quantum_Computing_and_the_Ising_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 4: Quantum Computing and the Ising Model

Ashley Dale

*This notebook also draws heavily from the following references: [1](https://quantumai.google/cirq/experiments/qaoa/qaoa_ising), [2](https://quantumai.google/cirq/simulate/qvm_builder_code), [3](https://quantumai.google/cirq/simulate/qvm_basic_example)*

----
In this notebook, you will learn the following concepts
- The *Quantum Approximate Optimization Algorithm*
- How to implement the Ising Model on a quantum circuit
- How to adjust a circuit for quantum hardware constraints
- How to simulate the quantum circuit to find the lowest energy state
- Midway's *Ten Principles of Data Visualization*

*Note: This notebook assumes the first half of an undergraduate course in quantum mechanics, and no prior knowledge about quantum computing.  It contains exactly the amount of quantum computing background required to implement the algorithm.  If you are unfamiliar with quantum computing, it is a good idea to check out the additional reading at the end of this notebook **before** attempting to write the code.*

#### Setup Python Environment

We will be using the `cirq` python package for this notebook

In [ ]:
try:
    import cirq
    import cirq_google
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq-google &> /dev/null
    print("installed cirq.")
    import cirq
    import cirq_google

try:
    import qsimcirq
except ImportError:
    print("installing qsimcirq...")
    !pip install --quiet qsimcirq &> /dev/null
    print(f"installed qsimcirq.")
    import qsimcirq

import time
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import copy
from tqdm import tqdm, trange

# Setup Quantum Virtual Machine

We will also be using a quantum virtual machine (QVM) to simulate our circuit.  This is a quantum computer approximator that is accurate to within the experimental error of a real quantum computer.  You can read more about it [here](https://quantumai.google/cirq/hardware/devices) and [here](https://blog.google/technology/research/our-new-quantum-virtual-machine-will-accelerate-research-and-help-people-learn-quantum-computing/).

Run the next code block to initialize the machine:

In [ ]:
processor_id = "rainbow"

# Instantiate an engine.
sim_engine = cirq_google.engine.create_default_noisy_quantum_virtual_machine(
    processor_id=processor_id, simulator_class=qsimcirq.QSimSimulator,
)
print(
    "Your quantum virtual machine",
    processor_id,
    "is ready, here is the qubit grid:",
    "\n========================\n",
)
sim_engine.verbose = False
print(sim_engine.get_processor(processor_id).get_device())

In this archicture, each $(i,j)$ pair on the grid represents a qubit.  Take a moment and make sure you can answer the following questions:
1. How many qubits are there?
2. What is the range for the row indices?  That is, what is the lowest value for $i$ and what is the highest index for $i$?
3. What is the range for the column indices?  That is, what is the lowest value for $j$ and what is the highest index for $j$?
4. If we try to superimpose an $N\times N$ lattice on this system, what is the maximum value for $N$?

# Background Information

## Recall from Notebook #2 ...

In the previous notebooks, we explored the Ising model Hamiltonian and found the energy of each spin in the Ising lattice was

<center>$\epsilon_i = -\frac{1}{2}J\sum_{j\in[1,4]} \sigma_i \sigma_j + \mu h \sigma_i$</center>

where
- $\mu$ is the atomic magnetic moment
- $h$ is an applied magnetic field
- the $\sigma$ operator "measures" the spin of the atom and returns value $+1$ for spin up or $-1$ for spin down.
- the $\frac{1}{2}$ factor ensures that we don't double count the contributions from neighboring atoms.
- $J$ is the coupling between nearest neighbors (the gray lines in Figure 2)
- The negative sign "$-$" in front of the $J$ means that this is a ferromagnetic system
- $j \in [1,4]$ means that only four nearest neighbors for each atom in Figure 2 will be included in the calculation

To get the total energy for the system, we sum over all the measured of the observed energies:

<center>$E = \sum_i \epsilon_i$</center>.

Then we used the Metropolis-Hastings Algorithm to minimize the energy of the system.

___
In the previous notebooks, we focused on the Metropolis-Hastings algorithm and its software execution by implementing the algorithm in Python and optimizing its timing.  Now, we will explore this same problem in a new context by changing the hardware from a standard computer to a quantum computer, and choosing an algorithm that lets us perform a similar kind of simulation.

## Quantum Computing Introduction



### Why use a quantum computer for this algorithm?

Quantum computers are good for quickly finding solutions to specific types of problems.  In general, these problems should have the following types of properties:

1. They require so much traditional computational power that it is not practical to solve them this way

2. The outcome of the problem should be a probability distribution, and not a single number (or state) every time

3. Currently, the best way available to find a solution is by using a [search algorithm](https://en.wikipedia.org/wiki/Search_algorithm)

The problem of finding the lowest energy configuration of an Ising Lattice always has properties (2) and (3) from the list above.  For property (2), the state of the system will be drawn from the Boltzmann distribution, and for property (3), the best way to find the outcome is through a type of random search.  If the Ising lattice is sufficiently large--that is, it has the same scale as an actual physical system with several moles of "spins"--then it will also have property (1).  That's why we can use this problem to test out a quantum computer.

### Quantum Circuits 101

A quantum circuit has two building blocks:

***qubit***: A small, quantum system that can have exactly two states $|0\rangle$ and $|1 \rangle$.  The qubit is prepared in a coherent superposition of these states:
<center> $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$ </center>
where $|\alpha|^2 + |\beta|^2 = 1$.

***quantum gate***: The specification of how a qubit's state will be changed.  It also allows two or more qubits to interact with eachother.  A quantum gate has the form of a unitary operator:

<center>$U(A, \omega) = e^{-i \pi \omega A /2}$</center>

where $A$ is the operator and $\omega$ is the phase of the wave.  

In this notebook, we will need the following operators to build quantum gates:

The Pauli $X$ operator for a single qubit:
<center> $X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$ </center>

The Pauli $Z$ operator for a single qubit:
<center> $Z = \begin{bmatrix} 1 & 0\\ 0 & -1 \\ \end{bmatrix}$</center>

The tensor product of two $Z$ operators for two qubits:
<center> $ZZ = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & -1 & 0 & 0 \\ 0 & 0 & -1 & 0 \\ 0 & 0 & 0 & 1\end{bmatrix}$ </center>

The Hadamard gate for a single qubit:
<center> $H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$ </center>

An ***operation*** is when a gate *acts* on a qubit or set of qubits.

[See this webpage](https://lewisla.gitbook.io/learning-quantum/quantum-circuits/single-qubit-gates) for some good visualizations of how the operators change the state of the qbit.

### Some Things to Understand

1. "Programming a quantum computer" means "building a quantum circuit that solves a specific problem".  Most computers you might be familiar with are in the category of [*Random-access machines*](), which distinguish between hardware and the software implemented on the hardware.  Quantum computers do not have this distinction.  The implementation of a quantum algorithm on hardware is a physical experiment.

2. It is not possible to measure a quantum circuit in a quantum computer while the quantum circuit is being operated.  Measuring the state of the circuit collapses the probabilities, so additional operations are not meaningful. **BUT** If we are simulating a quantum circuit on a classical computer, it is possible to take advantage of the classical behavior and make measurements during the simulation.  Since we are using a quantum virtual machine, we will respect the physical aspects of the hardware and only measure the circuit at the end of the experiment.

## The Quantum Approximate Optimization Algorithm (QAOA)

This algorithm was published by Farhi, Goldstone, and Gutmann in 2014; you can [read the original paper here](https://arxiv.org/abs/1411.4028).  It is designed to approximate a solution for the problem of how to choose the best combination of objects from some larger set of objects.  **For our Ising lattice, the "larger set" of objects is the set of all possible combinations of spin states in the lattice, and the "best combination" is the set of spin states that minimize the total energy of the system as calculated by the Ising Hamiltonian.**  

Other common example problems for this algorithm would be the [Traveling Salesman Problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem) or the [Knapsack Problem](https://en.wikipedia.org/wiki/Knapsack_problem).  In fact, this type of combinatorial optimization problem is fairly common throughout physics, [biology](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2443096/), [chemistry](https://pennylane.ai/qml/demos/tutorial_mol_geo_opt/), and [economics](https://iopscience.iop.org/article/10.1088/1742-6596/820/1/012028), which is why quantum computers are an exciting technology.
___
The **Quantum Approximate Optimization Algorithm (QAOA)** steps can be summarized as follows:


0. Make a circuit of $n$ qubits prepared in state $|0\rangle$.

1. Use a layer of Hadamard gates to put the qubits in a superposition of $|\psi\rangle = \frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}|1\rangle$

2. Add a layer of $C$ gates to create a distribution that matches the objective function by changing the *phase* of the qubits

    >We haven't defined the objective function or the $C$ gates yet, but the idea here is that the qubits are entangled, and then prepared in a state that matches the energy distribution in which we are interested.

3. Add a layer of $B$ gates to create a "mixing" of probabilties so that the system is no longer in an eigenstate.  

    >The important thing here is that the $B$ gate operator and the $C$ gate operator do not commute.  If the two do commute, then taking a measurement returns an eigenstate of the system.

4. Continue adding paired layers of $C$ and $B$ gates as useful

    >This is analgous to how many steps performed in the Metropolis-Hastings algorithm.  The exact number of QAOA layers needs to be optimized.

5. Measure the outcome and calculate the energy of the distribution

    >This is analgous to calculating the energy $E$ in the Metropolis-Hastings algorithm.  




### The $C$ and $B$ Gates



First, we need to understand how to build the unitary operators $C$ and $B$ as quantum gates.

**Starting with the $C$ gate:**

Recall the Ising Hamiltonian:

<center>$\epsilon_i = -\frac{1}{2}J\sum_{j\in[1,4]} \sigma_i \sigma_j + \mu h \sigma_i$</center>

We need to re-write the this as a unitary operator.

To make things easy on ourselves, set all of the constants except the magnetic field $h$ equal to 1:

<center>$\epsilon_i = - \sum_{j\in[1,4]} \sigma_i \sigma_j +  h \sigma_i$</center>

Now, recall that the $\sigma$ operator "measures" the state of an Ising spin and returns $\pm 1$; we can make a quantum gate equivalent by replacing the $\sigma$ operator with the Pauli $Z$ operator

<center> $E_i = -\sum_{j\in [1,4]} Z_iZ_j + hZ_i $</center>

Next, create the unitary operator $C$ where we will use the Hamiltonian $E_i$ as the $A$ operator:

<center>$C(E_i, \omega) = e^{i \pi \omega E_i /2} = e^{i \pi \omega(-\sum_{j\in [1,4]} Z_iZ_j + hZ_i)/2 }$</center>

By using the properties of exponents $e^Ae^B = e^{A+B}$, we can rewrite the above as
<center>$C(E, \omega) = \prod_{j\in [1,4]}e^{-i\pi\omega Z_iZ_j/2} \prod_ie^{-i \pi \omega h Z_i /2}$</center>.

The $\omega$ in this expression is a constant representing the phase difference between the two qubits $(i,j)$ that we will define in the code; to explore how to determine optimal values for $\omega$ [see this tutorial](https://quantumai.google/cirq/experiments/qaoa/qaoa_ising).  

The only thing left is to create matrix forms of the two terms in $C(E,\omega)$, and we can do this using the Pauli $Z$ operators defined earlier and the [matrix property of exponentials](https://sites.millersville.edu/bikenaga/linear-algebra/matrix-exponential/matrix-exponential.html):

<center>$e^{-i \pi \omega Z_iZ_j/2} = \begin{bmatrix} e^{-i \pi \omega/2} & 0 & 0 & 0 \\ 0 & e^{i \pi \omega/2}& 0 & 0 \\ 0 & 0 & e^{i \pi \omega/2} & 0 \\ 0 & 0 & 0 & e^{-i \pi \omega/2} \end{bmatrix}$</center>

and

<center>$e^{-i \pi \omega h Z_i /2} = \begin{bmatrix} e^{-i \pi \omega h/2} & 0 \\0 & e^{i \pi \omega h/2}\end{bmatrix}$</center>.

**Now the $B$ gate:**

Since the goal of the $B$ gate is to move the system out of an eigenstate, we can choose any operator that does not commute with the $Z$ Pauli matrix; the Pauli $X$ matrix works well for this role.  Since the $B$ gate is also unitary, we can simply write the $B$ gate as:

<center>$B(X, \beta) = e^{i \pi \beta X/2}$</center>

which in matrix form is

<center>$B(X, \beta) = \begin{bmatrix}0&e^{i\pi\beta/2}\\e^{i \pi \beta/2}& 0\end{bmatrix}$</center>


The whole algorithm for two qubits can be summarized as shown below.  The QAOA layers are repeated $p$ times, where $p$ is the number of QAOA layers required to get good results.  The $C$-gate is shown as a composite of $Z$ gates.  The system measurement (not shown) made after the end of the last QAOA layer.

<img src=https://raw.githubusercontent.com/GDS-Education-Community-of-Practice/DSECOP/dale/Connecting_MonteCarlo_to_ModernAI/qaoa.png alt="Summary of the Quantum Approximate Optimization Algorithm.  Source: Own Work" align="center">


### Implementation in Python
Now we are going to actually implement the algorithm.  Keep the six steps from above in mind, and run the next cell to see the device again:

In [ ]:
print(sim_engine.get_processor(processor_id).get_device())

#### QAOA Step 0. Define Qubits

We are going to choose the center qubit at location $(4,2)$, and the four neighbors at $(3,2), (4, 1), (5,2), (4,3)$ as our Ising lattice.  The easiest way to manipulate the qubits is to make a list of them:

In [ ]:
q0 = cirq.GridQubit(4, 2)
q1 = cirq.GridQubit(3, 2)
q2 = cirq.GridQubit(4, 1)
q3 = cirq.GridQubit(4, 3)
q4 = cirq.GridQubit(5, 2)
qubits = [q0, q1, q2, q3, q4]

In [ ]:
# Initialize the constants needed for the unitary operators
# These values are reused from the Google AI Tutorial optimizing the QAOA
omega = 0.2
beta = 0.25
h = 0.5

#### QAOA Step 1. Hadamard Gates

In [ ]:
# Start in the H|0> state.
qaoa = cirq.Circuit(cirq.H(q0))
qaoa.append(cirq.H(q1))
qaoa.append(cirq.H(q2))
qaoa.append(cirq.H(q3))
qaoa.append(cirq.H(q4))

#### QAOA Step 2. C-Gate

In [ ]:
# Now we can add the C-gate in two steps
# First we add the ZZ-gate
qaoa.append(cirq.ZZ(q0, q1) ** omega)
qaoa.append(cirq.ZZ(q0, q2) ** omega)
qaoa.append(cirq.ZZ(q0, q3) ** omega)
qaoa.append(cirq.ZZ(q0, q4) ** omega)

# Second, we add the Z-gate
qaoa.append(cirq.Z(q0) ** (omega * h))
qaoa.append(cirq.Z(q1) ** (omega * h))
qaoa.append(cirq.Z(q2) ** (omega * h))
qaoa.append(cirq.Z(q3) ** (omega * h))
qaoa.append(cirq.Z(q4) ** (omega * h))

#### QAOA Step 3. B-Gate

In [ ]:
# Next we can add the B-gate
qaoa.append(cirq.X(q0) ** beta)
qaoa.append(cirq.X(q1) ** beta)
qaoa.append(cirq.X(q2) ** beta)
qaoa.append(cirq.X(q3) ** beta)
qaoa.append(cirq.X(q4) ** beta)

#### QAOA Step 4. Add More Layers

The best way to do this is to create a layer generator which can be called in a loop.  To see a good example of how to write layer generators, check out the [tutorial here](https://quantumai.google/cirq/experiments/qaoa/qaoa_ising).  

We will continue past this step for now to keep the implementation simple.

#### QAOA Step 5. Measure

In [ ]:
# Finally, we add measurement at the end of the circuit
qaoa.append(cirq.measure(*qubits, key='out'))

In [ ]:
# Visualize the circuit by running this cell
print(qaoa)

----

#### What do we expect to measure?

Since this is a probabilistic measurement, we want to have an intuition of how the results will be returned and what they will mean:

First, if you inspect the `print(qaoa)` output in the cell above, you will see that there are measurements applied to each qubit.  This means that for each repeated simulation, we will get a vector that belongs to the following set of outcomes:

---
*Let q0 be the center qubit of our five connected qubits*

|$\mid$ q0, q1, q2, q3, q4 $\rangle$ | State Description
|---|---
|$\mid$00000$\rangle$|All qubits are in the ground state
|$\mid$00001$\rangle$, $\mid$00010$\rangle$, $\mid$00100$\rangle$, $\mid$01000$\rangle$, $\mid$10000$\rangle$|Only the $h\sigma$ in the Ising Hamiltonian contributes to the energy $E$
|$\mid$00011$\rangle$, $\mid$00101$\rangle$, $\mid$01001$\rangle$, $\mid$10001$\rangle$, ..., $\mid$01100$\rangle$ | 2 of the neighboring qubits are in "spin up"
|$\mid$00111$\rangle$, $\mid$01110$\rangle$|3 of the neighboring qubits are "spin up"
|$\mid$01111$\rangle$| 4 of the neighboring qubits are "spin up"
|$\mid$10001$\rangle$, $\mid$10010$\rangle$, $\mid$10100$\rangle$, $\mid$ 11000 $\rangle$ | The center qubit and 1 neighbor
|$\mid$10011$\rangle$, $\mid$10101$\rangle$, $\mid$11001$\rangle$, ..., $\mid$11100$\rangle$| The center qubit and 2 neighbors
|$\mid$10111$\rangle$, $\mid$11110$\rangle$ |The center qubit and 3 neighbors
|$\mid$11111$\rangle$| All qubits are "spin up"

---
Every time we run the experiment, we should get one of the outcomes from the above table as a result.  However, the *probability* of getting a particular vector as a result depends on the parameters $\omega$, $\beta$ and $h$; this is analogous to the way the outcome of the Metropolis-Hastings algorithm depends on the thermodynamic temperature $k_BT$.  

### Execution on Quantum Virtual Machine

To execute this on a quantum virtual machine (QVM), we have to do the following steps:
1. Transform the circuit to use gates which are available on the QVM.  
>Just because we can theoretically define something doesn't mean that it is actually possible on the hardware.

2. Simulate the circuit on the QVM we defined earlier

In [ ]:
# 1. Transform the circuit

# Convert the gates in the circuit to the "SqrtIswap" gateset, which this QVM uses.
translated_circuit = cirq.optimize_for_target_gateset(
    qaoa, context=cirq.TransformerContext(deep=True), gateset=cirq.SqrtIswapTargetGateset()
)
print(translated_circuit)

In [ ]:
#2. Simulate the circuit

reps = 3000 # How many times to repeat the experiment

start = time.time() # Start a timer

## --> The actual experiment is executed here:
results = sim_engine.get_sampler(processor_id).run(translated_circuit, repetitions=reps)

elapsed = time.time() - start # End the timer

print('Circuit successfully executed on your quantum virtual machine', processor_id)
print(f'QVM runtime: {elapsed:.04g}s ({reps} reps)')
print('You can now print or plot "results"')

We can see that one vector of length $1\times5$ has been returned for each of the 3000 simulations by checking the shape of the returned results for the `M('out')` value of the quantum circuit.  This makes sense, because we have 5 qubits in our lattice.

In [ ]:
results.records['out'].shape

## Visualizing the Results

[Data Visualization is considered a very important part of data science](https://www.coursera.org/articles/data-visualization); some professionals specialize in it.  The ability to communicate results clearly and effectively requires practice, and many resources are available to help.

A good approach for developing figures comes from [Midway's 10 Principles of Effective Data Visualization](https://www.sciencedirect.com/science/article/pii/S2666389920301896).  Midway's principles can be briefly summarized as follows:

1. Diagram the figure first
2. Use the right software for the job
3. Use the right kind of plot and show data
4. Colors always mean something
5. Include uncertainty (like error bars)
6. Use subfigures when possible
7. "Data" and "modeling the data" are different things
8. Simple visuals, detailed captions
9. Consider an infographic
10. Get a second opinion

In the following exercises, we will convert the raw data returned by the quantum circuit into something meaningful.

#Exercises


### Programming Exercise 1. Implement the QAOA with 2 Layers and run a new experiment

Copy-paste the code above in the correct place below to make sure you understand each step.  Feel free to make the implementation cleaner by implementing for-loops and defining functions.  **Print the circuit at the end to show the two QAOA layers.**

We will reuse the same qubit definitions from above; step 0 and step 1 are already completed for you.

After defining the circuit, rerun the same experiment again.  **You may also define an experiment function to make the code easier to reuse.**

In [ ]:
# Step 0. is already completed for you
omega = 0.2
beta = 0.25
h = 0.5
print(qubits)

In [ ]:
# Step 1. Add Hadamard gates:
my_circuit = cirq.Circuit(cirq.H(q0))
for qubit in [q1, q2, q3, q4]:
    my_circuit.append(cirq.H(qubit))

In [ ]:
# Step 2. Add C-gate layer by copy-pasting code from above here:

# First we add the ZZ-gate

# Second, we add the Z-gate


In [ ]:
# Step 3. Add B-gate layer by copy-pasting code from above here:


In [ ]:
# Step 4. Add a second C-layer followed by a second B-layer
# First we add the ZZ-gate


# Second, we add the Z-gate


In [ ]:
# Step 5. Add a measurement at the end of the circuit


In [ ]:
# Run this command to visualize your circuit
print(my_circuit)

Run your 2-layer circuit by executing the following code cells:

In [ ]:
# Convert the gates in the circuit to the "SqrtIswap" gateset, which the device uses.
device_ready_circuit = cirq.optimize_for_target_gateset(
    my_circuit,
    context=cirq.TransformerContext(deep=True),
    gateset=cirq.SqrtIswapTargetGateset()
)
print(device_ready_circuit)

reps = 50000 # How many times to repeat the experiment
start = time.time() # Start a timer
## --> The actual experiment is executed here:
results = sim_engine.get_sampler(processor_id).run(translated_circuit, repetitions=reps)
elapsed = time.time() - start # End the timer

print('Circuit successfully executed on your quantum virtual machine', processor_id)
print(f'QVM runtime: {elapsed:.04g}s ({reps} reps)')
print('You can now print or plot "results"')

## Programming Exercise 2. Calculate the Energy

To determine the energy of the system, we need to use the Ising Hamiltonian with the constants specified.

Recall that

<center> $E_i = -\sum_{j\in [1,4]} Z_iZ_j + hZ_i $</center>

.  This can be expanded further as:

<center>$E = \left(-1\right)\left( q_0q_1 + q_0q_2 + q_0q_3 + q_0q_4\right) + h[q_0 + q_1 + q_2 + q_3 + q_4] $</center>

where $q_i$ represents the value $\pm1$ of the $ith$ qubit, and $q_0$ is the value of the center qubit at location $(4, 2)$.


- Define a new function "calculate_energy" (outlined for you below) that takes as input the bit string returned at the end of a measurement, the applied magnetic field, and returns the total energy $E$ (defined above) of the **qubit at location (4, 2)** at the end of the measurement.

In [ ]:
def calculate_energy(qubits, h, qidx=2):

    # Since the circuit only returns 0 and 1, we need to convert the 0s to -1
    # using an array mask

    qubits[qubits == 0] = -1

    # The indices for the qubits is as follows:
    # q0 = qubits[qidx]
    # q1 = qubits[0]
    # q2 = qubits[1]
    # q3 = qubits[3]
    # q4 = qubits[4]

    # Implement the equation from above here
    E =

    return E

Now, repeat the experiment for a range of $\omega$ and $h$ values and calculate the energy for each experiment:

In [ ]:
### Here is some code to help you run your experiment

# Parameter selection
num_pts = 10
omega=np.linspace(0, 1, num_pts)
beta=1
h=np.linspace(-1, 1, num_pts)
reps=5000
E_average = []
E_uncertainty = []

# Collect your data here:
for o in omega:

    tmp_E = []
    tmp_s = []

    for h_field in h:

        # Either call your experiment function from above, or copy-paste
        # the needed code here.  Save your results to the 'results' variable

        results =
        my_results = np.squeeze(copy.deepcopy(results.records['out']))

        total_energies = []
        total_states = []

        for idx in range(reps):
            E, s = calculate_energy(my_results[idx], h_field)
            total_energies.append(E)
            total_states.append(s)

        tmp_E.append(np.mean(total_energies))
        tmp_s.append(np.std(total_energies))

    E_average.append(tmp_E)
    E_uncertainty.append(tmp_s)

### Optional: Implement Gradient Descent

Gradient descent may be used to select the optimal approximate parameters instead of performing a grid search in the manner above.  [See the tutorial here](https://quantumai.google/cirq/experiments/qaoa/qaoa_ising), and implement gradient descent to approximate the paramters.

## Programming Exercise 3. Visualize Results

Using Midway's 10 Principles of Effective Data Visualization, create a three panel figure that explains:
1. The Quantum Approximate Optimization Algorithm with two layers of [C,B] gates
2. The Energy Distribution returned for a single set of parameters $\omega$, $\beta$, and $h$
3. How the uncertainty in the energy expectation values changes with the hyperparameters

You will want to repeat your code in **Programming Exercises 1 & 2** multiple times to obtain results for your third panel.  To make this easier, consider wrapping the code into a single function.

### Step 1. "Diagram First"
Answer the following question with a few sentences below: Given the figure description, what will a viewer be able to understand about your work?

*Enter your answer here*


### Step 2. "The Right Software"

For the panel explaining the QAOA, you can use any graphical design software you choose.  Some suggestions are:

- Microsoft Power Point ([free account online](https://www.microsoft.com/en-us/microsoft-365/free-office-online-for-the-web))
- [Google Slides](https://www.google.com/slides/about/)
- [draw.io](https://app.diagrams.net/)

It is also recommended to do a Google Image search and see what other graphical representations of the algorithm look like.  You should not plagiarize someone elses' work, but you will want to think about what things are good to leave out or include--inspecting what others visualized can be helpful in this process.

For the remaining two panels, take a look at these python packages:

- [MatPlotLib.PyPlot](https://matplotlib.org/stable/gallery/index.html)
- [SeaBorn](https://seaborn.pydata.org/examples/index.html)
- [QuTip - visualize quantum phenomena](https://qutip.org/qutip-tutorials/#visualizations), e.g. [Bloch Sphere Animations](https://nbviewer.org/urls/qutip.org/qutip-tutorials/tutorials-v4/visualization/bloch-sphere-animation.ipynb), [Energy Level Diagrams](https://nbviewer.org/urls/qutip.org/qutip-tutorials/tutorials-v4/visualization/energy-levels.ipynb)
- [cirq - state histograms](https://quantumai.google/cirq/simulate/state_histograms)
- [cirq - heat map](https://quantumai.google/cirq/noise/heatmaps)

---

Answer the following question with a sentence and/or two below: Which software or python packages did you pick, and why?

*Enter your answer here*

### Step 3. "Geometry and Data"

The goal is to present the most information (aka "data") using the least amount of "ink".  This can be achieved by carefully considering the best way of

A "geometry" in this context is the shape of your data visualization on the page.  Geometries can be divided into several categories:

- *amounts or comparisons*: Only use when there is no distribution and no uncertainty.  Examples include bar graphs, cleveland dot plots, heat maps.  

- *compositions or proportions*: Use to show how fractional components relate to each other.  Examples include stacked or clustered bar charts, stacked density plots, mosaic plots, treemaps.

- *distributions*: Specifically, probability distributions for different parameters.  Examples include box plot, histogram, violin plots, and density plots.

- *relationships*: Vary one parameter and observe a change in a second parameter.  Most common example is a scatter plot with layered information in the marker style, color, size.  Extends to line plots.

Finally, including the raw data on the plot alongside the analysis is preferred.

---

Answer the following question with a sentence and/or two below: What geometries will you choose for panels 2 and 3?  Why?

*Enter your answer here*


### Step 4. "Colors"

[See here](https://matplotlib.org/stable/users/explain/colors/colormaps.html#colormaps) for a good in-depth overview of this topic.

[Color maps](https://seaborn.pydata.org/tutorial/color_palettes.html) come in three different kinds:

- *sequential*: the colors go from dark to light.  The range of colors typically implies a range of numerical values where increasing value is associated with increasing darkness.
- *diverging*: Two sequential maps are connected with the light color in the middle, representing two numerical extremes.
- *qualitative*: Different colors indicate different groups.

It is worth considering whether your readers may be [colorblind](https://www.color-blindness.com/), and choosing a colormap which is inclusive.

---

Answer the following question with a sentence and/or two below: What kind of colormap is appropriate for each panel, and why?

*Enter your answer here*


### Step 5. "Uncertainty"

Midway presents two challenges to representing uncertainty: failure to show any uncertainty and misrepresentation or misinterpretation of uncertainty.  To overcome these challenges, it is worth considering the following:

**Implicit Representation**

In an implicit representation, the uncertainty is already included in the data representation, and no significant effort needs to be made to further emphasize it.  This is generally true when plotting data distributions.  Examples include a noisy [scatter plot](https://datavizcatalogue.com/methods/scatterplot.html), a [box plot](https://datavizcatalogue.com/methods/box_plot.html), a [violin plot](https://datavizcatalogue.com/methods/violin_plot.html), or a [density plot](https://datavizcatalogue.com/methods/density_plot.html).

**Explicit Representation**  

In an explicit representation, the uncertainty is added to the figure as an additional geometry, usually in the form of [error bars](https://datavizcatalogue.com/methods/error_bars.html) for discrete uncertainty values or a shaded [uncertainty region](https://blogs.sas.com/content/iml/2020/10/14/continuous-band-plot-uncertainty-predictions.html) for continuous uncertainty values.  Explicit representations are commonly required when presenting averaged data.

An excellent reference for visualizing uncertainty may be found [here](https://clauswilke.com/dataviz/visualizing-uncertainty.html).

---

Answer the following question below: Given the uncertainty data you have already calculated, what is the most appropriate representation?  

*Enter your answer here*


### Step 6. "Subfigures"

Subfigures are used to connect a sequence of otherwise independent figures into a sequence.  The sequence implies the reader should not interpret each figure independently, but instead interpret each subfigure in the context of all other subfigures in the image.

Here, you are already creating three subfigures in a single figure.

### Step 7. "Modeling the data"

Consider the following figure:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(6, 3))
x = np.linspace(-1, 1, 50)
noise = np.random.uniform(size=50)
ax[0].scatter(x, x*noise, s=10)
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[0].set_title("(a) Raw Data")
ax[1].scatter(x, x*noise, s=10)
ax[1].plot(x,x, c='r')
ax[1].set_xlabel('x')
ax[1].set_ylabel('y')
ax[1].set_title('(b) Data with Model')
fig.tight_layout()

In (a) the raw data is presented and in (b), the raw data is presented with a model (shown as a red line).  **The model is not part of the data.**  Plotting the model in addition to the data is encourages the reader to visually interpret the data in the context of additional information.  Here, we can see that while the data definitely has a linear trend, the uncertainty in the data prevents the model from being a perfect fit.


### Step 8. "Captioning the results"

The text accompanying a figure should enable it to be understood without additional context.  Two kinds of text can accompany a figure or image:

1.  **Caption**.  The caption should summarize the main point of the figure and include any noteworthy details.

1. **Alternative Text or "Alt Text"**.  In a digital context, the alt text is a brief description of an image in case the original image is not viewable.  This is an important inclusivity step that facilitates assistive technology such as screen readers commonly used by individuals with low vision.  It can also affect how your figure is found during a web search.  Although there may not be an obvious option for alt-text in a formal research publication, any HTML environment and presentation software such as MS Power Point have this option.

---
In the space below, write a caption and an alternative text for your figure that has the following information:

1. A brief statement of the figure's purpose
1. A description of the information for each sub figure
1. An alt text statement to accompany the figure and caption

### Resources

[CSUN Best Practices for Accessible Images](https://www.csun.edu/universal-design-center/best-practices-accessible-images)

[Guide to writing alt text and accessible captions](https://mashable.com/article/how-to-write-accessible-image-captions)

[Why alt text is important for search engine optimization](https://www.innovationvisual.com/knowledge/why-image-alt-text-is-important-for-seo)



*Enter your answer here*

### Steps 9. and 10.

#### 9. Infographics:

Infographics can be useful for visualizing an entire process.  They are similar to a research poster, but usually intended for a non-technical audience.  For lots of good examples, see [Daily Infographic](https://dailyinfographic.com/).

#### 10. Get a second opinion

>Enough said.

In [ ]:
### WORK SPACE: Write any necessary code for your figure here


# Question 1. Pauli Z operator as Ising Spin Operator


Let $|0\rangle = \begin{bmatrix}1\\0\end{bmatrix}$ be the $(+1)$ Ising state "spin up", and $|1\rangle = \begin{bmatrix}0\\1\end{bmatrix}$ be the $(-1)$ Ising state "spin down".  Show that for Pauli matrix $Z=\begin{bmatrix}1&0\\0&-1\end{bmatrix}$
<center>$Z|0\rangle = \begin{bmatrix}1\\0\end{bmatrix}$</center>

<center>$Z|1\rangle = \begin{bmatrix}0\\-1\end{bmatrix}$</center>


*Enter your answer here*

# Question 2. Non-Commuting Operators

Show that the $X$ operator and the $Z$ operator do not commute:

$X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$

$Z=\begin{bmatrix}1&0\\0&-1\end{bmatrix}$

*Enter your answer here*

# Additional Reading

The QAOA is a very powerful algorithm, and this notebook does only the simplest implementation.  

1. [See here](https://quantumai.google/cirq/experiments/qaoa) for a more thorough discussion of how to tune the paramters and create a more rigorous implementation.
2. [Quantum Monte Carlo Markov Chain](https://www.nature.com/articles/s41586-023-06095-4)
3. [Also here](https://research.ibm.com/blog/quantum-markov-chain-monte-carlo#-fn-1)
4. [Role of Mixing Layer](https://quantumcomputing.stackexchange.com/questions/17555/whats-the-role-of-mixer-in-qaoa)

## Data visualization Resources

1. [Free Textbook: Fundamentals of Data Visualization](https://clauswilke.com/dataviz/)
1. [Data Visualization Catalog](https://datavizcatalogue.com/index.html)
1. [Visualizing Errors in Python](https://jakevdp.github.io/PythonDataScienceHandbook/04.03-errorbars.html)

## Implementation on Real Quantum Computer

- At time of writing (12/2023), IBM offers access to their quantum computer through [Qiskit](https://www.ibm.com/quantum/qiskit).  See if you can implement the same algorithm on a different platform.
